In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
from nltk.corpus import wordnet
syns = wordnet.synsets("program")

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import re
import unicodedata
import sys

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import urllib.request
import urllib.response
import sys
import os, glob
import http.client, urllib
import json
import http.client, urllib.request, urllib.parse, urllib.error, base64

# Replace the accessKey string value with your valid access key.
accessKey = '82e4612615634c398bfd26e7d6327833'
url = 'westcentralus.api.cognitive.microsoft.com'
path = '/text/analytics/v2.0/keyPhrases'

def extract_keywords(body):
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = http.client.HTTPSConnection(url)
    body_req = {'documents':[{'language': 'en', 'id': 1, 'text': body}]}
    body_json = json.dumps(body_req)
    conn.request ("POST", path, body_json, headers)
    response = conn.getresponse ()
    string = response.read().decode('utf-8')
    json_obj = json.loads(string)
    print("json_obj: ", json_obj)
    keyphrases_list = ((json_obj['documents'])[0])['keyPhrases']
    print("keyphrases_list: ", keyphrases_list)
    return keyphrases_list

In [3]:
# pronto needed to prase obo files
import pronto
ont = pronto.Ontology('https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/doid.obo')

# medical conditions training data
training_data = []

# adds every medical condition with listed symptoms
for term in ont:
    if "has_symptom" in term.desc:
        training_data.append({"class":term.name, "sentence":term.desc})
print ("%s total classes in training data" % len(training_data))

# chooses random 100 medical conditions with symptoms
training_data = random.sample(training_data, 25)
print ("%s randomly chosen classes in training data" % len(training_data))
for term in training_data:
    print(term["class"] + " | " + term["sentence"] )

358 total classes in training data
25 randomly chosen classes in training data
bovine hemorrhagic syndrome | A viral infectious disease that results_in infection in cattle, has_material_basis_in Bovine viral diarrhea virus 2, which is transmitted_by ingestion of infected food, or transmitted_by nasal secretions. The infection has_symptom low grade fever, has_symptom bloody diarrhea, and has_symptom bleeding in the eyes.
HIV leukoencephalopathy | A HIV encephalopathy that results_in inflammation located_in white matter of neuraxis, has_material_basis_in Human immunodeficiency virus 1 or has_material_basis_in Human immunodeficiency virus 2. The infection has_symptom weakness, has_symptom paralysis, has_symptom vision loss, has_symptom impaired speech, and has_symptom cognitive deterioration.
tuberculous epididymitis | An urogenital tuberculosis that is located_in epididymis, has_symptom pain and has_symptom scrotal swelling.
AIDS dementia complex | A HIV encephalopathy that results_in in

In [4]:
# a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

# method to remove punctuations from sentences.
# def remove_punctuation(text):
#     return text.translate(tbl)

# method to slice strings
def slicer(my_str,sub):
    index=my_str.find(sub)
    if index !=-1 :
         return my_str[index:] 
    else :
         raise Exception('Sub string not found!')

# find synonyms of all the words in a sentence
def find_syns(sentence):
    words = []
    for word in sentence:
        if word != '':
            synonyms = []
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    if l.name() not in synonyms and l.name() != word:
                        synonyms.append(l.name())
            word = (word + ' ' + ' '.join(synonyms)).strip()
            words.append(word)
    return words

# clean all synonyms and words
def clean_syns(arg):
    clean_synonyms = []
    for syn_set in synonyms:
        syn_set = syn_set.replace("_", " ")
        syn_set = ' '.join(s for s in syn_set.split() if not any(c.isdigit() for c in s))
        syn_set = re.sub(r'\d+', '', syn_set)
        clean_synonyms.append(syn_set)
    #print("CLEAN SYNONYMS: ",clean_synonyms)
    return clean_synonyms

In [5]:
import string

words = []
words_total = []
classes = []
classes_details = {}
documents = []
ignore_words = ['?', 'hassymptom']

# loop through each description per symptom
for pattern in training_data:
    #print("CLASS: " + pattern['class'])
    #print("DESCRIPTION: " + pattern['sentence'])
    #strip punctuations, numbers, and words containing numbers
    stripped = slicer(pattern['sentence'],"has_symptom")
    word_list = stripped.split()
    word_list_new = []
    for word in word_list:
        if '_' in word:
            word = 'and'
        word_list_new.append(word)
    no_punct1 = ' '.join(s for s in word_list_new)
    no_nums1 = ' '.join(s for s in no_punct1.split() if not any(c.isdigit() for c in s))
    no_nums2 = re.sub(r'\d+', '', no_nums1)
    #print("BODY: " + no_nums2)
    keywords = extract_keywords(no_nums2)
    
    #find synonyms of each word in description
    synonyms = find_syns(keywords)
    synonyms_clean = clean_syns(synonyms)
    
    # tokenize each word in the description
    w_list = []
    for syn_set in synonyms_clean:
        w_token = nltk.word_tokenize(syn_set)
        w = [stemmer.stem(i.lower()) for i in w_token if i not in ignore_words]
        w = sorted(list(set(w)))
    
        # strip duplicates
        for item in w:
            if not item in words_total and item != '':
                words_total.append(item.strip())
            else:
                w.remove(item)
    
        w_syn = ' '.join(w)
        w_list.append(w_syn)
    
    #add each string to words
    for item in w_list:
        if not item in words and item != '':
            #print("Item is in words array already.")
            words.append(item.strip())
     
    # add to documents in our corpus
    documents.append((w_list, pattern['class']))
    
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])
        classes_details[pattern['class']] = pattern["sentence"].replace("_", " ")
        
#remove duplicates in words
words = sorted(list(set(words)))

# remove duplicates in classes
classes = sorted(list(set(classes)))

print (len(documents), "documents", documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
print ("classes_details", classes_details)

json_obj:  {'errors': [], 'documents': [{'id': '1', 'keyPhrases': ['bloody diarrhea', 'low grade fever', 'bleeding']}]}
keyphrases_list:  ['bloody diarrhea', 'low grade fever', 'bleeding']
json_obj:  {'errors': [], 'documents': [{'id': '1', 'keyPhrases': ['impaired speech', 'vision loss', 'paralysis', 'cognitive deterioration']}]}
keyphrases_list:  ['impaired speech', 'vision loss', 'paralysis', 'cognitive deterioration']
json_obj:  {'errors': [], 'documents': [{'id': '1', 'keyPhrases': ['pain']}]}
keyphrases_list:  ['pain']
json_obj:  {'errors': [], 'documents': [{'id': '1', 'keyPhrases': ['motor dysfunction', 'behavioral change', 'speech problems']}]}
keyphrases_list:  ['motor dysfunction', 'behavioral change', 'speech problems']
json_obj:  {'errors': [], 'documents': [{'id': '1', 'keyPhrases': ['diarrhea', 'gastointestinal bleeding']}]}
keyphrases_list:  ['diarrhea', 'gastointestinal bleeding']
json_obj:  {'errors': [], 'documents': [{'id': '1', 'keyPhrases': ['head tilt', 'incoordi

In [6]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print("TOKENIZED WORDS: ",pattern_words)
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

TOKENIZED WORDS:  ['bloody diarrhe', 'fev grad low', 'blee blood haemorrh hemorrh leech phlebotomise phlebotomize run shed']
TOKENIZED WORDS:  ['impair speech', 'loss vis', 'palsy paralys', 'cognit detery']
TOKENIZED WORDS:  ['ail anct annoy ass both hurt in inflict neck nuis pain sens the troubl']
TOKENIZED WORDS:  ['dysfunct mot', 'behavy chang', 'problem']
TOKENIZED WORDS:  ['bowel diarrhoe loos of', 'gastointestin']
TOKENIZED WORDS:  ['head tilt', 'incoordin', 'atax ataxy dyssynerg', 'musc trem', 'pares']
TOKENIZED WORDS:  ['discharg pus', 'eyelid lid palpebr', 'conjunctiv swol']
TOKENIZED WORDS:  ['pulmon ral', 'petech', 'edem periph', 'les skin']
TOKENIZED WORDS:  ['nause sick', 'photalg photophob', 'ach cephalalg concern headach vex worry', 'chil cool down frisson gelid icy pal quiv shiv shud thrill tingl']
TOKENIZED WORDS:  ['expery traum', 'acut disord stress', 'calend mon', 'fright thought', 'bad dream', 'avoid away dodg shun turn', 'asd']
TOKENIZED WORDS:  ['stiff', 'cephala

In [7]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=10000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 39999  | total loss: 0.41337 | time: 0.014s
| Adam | epoch: 10000 | loss: 0.41337 - acc: 0.9603 -- iter: 24/25
Training Step: 40000  | total loss: 0.37460 | time: 0.017s
| Adam | epoch: 10000 | loss: 0.37460 - acc: 0.9642 -- iter: 25/25
--
INFO:tensorflow:C:\Users\Steven\Documents\HTN Project NEW\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y, 'classes_details':classes_details}, open( "training_data", "wb" ) )